In [13]:
using PlotlyJS
using PyCall
using Colors
using OSMToolset
using OpenStreetMapX, OpenStreetMapXPlot
using Downloads
using GZip
using DataFrames
using Statistics
using Plots
using Downloads
using HTTP
using JSON
using EzXML
include("analyse.jl")
include("prepare_data.jl")

extract_points2 (generic function with 1 method)

In [14]:
flm = pyimport("folium")

PyObject <module 'folium' from '/home/adamkas/.local/lib/python3.12/site-packages/folium/__init__.py'>

In [15]:
download_city_with_bounds("Kraków","8")

"The file is already downloaded"

In [16]:
krakow_df = get_POI("Kraków.osm",nothing,"Kraków.csv")

Row,elemtype,elemid,nodeid,lat,lon,key,value,group,influence,range
,Symbol,Int64,Int64,Float64,Float64,String,String,Symbol,Float64,Float64
1,node,26860287,26860287,50.0155,19.9245,tourism,camp_site,leisure,5.0,1500.0
2,node,29783000,29783000,50.0849,20.0204,public_transport,stop_position,transport,5.0,300.0
3,node,30371892,30371892,50.0364,19.8253,public_transport,stop_position,transport,5.0,300.0
4,node,30372002,30372002,50.0488,19.9016,public_transport,stop_position,transport,5.0,300.0
5,node,32063299,32063299,50.0952,19.989,public_transport,stop_position,transport,5.0,300.0
6,node,34806198,34806198,50.072,20.1054,public_transport,stop_position,transport,5.0,300.0
7,node,41943557,41943557,50.0836,19.8598,public_transport,station,transport,5.0,300.0
8,node,41943557,41943557,50.0836,19.8598,railway,station,transport,10.0,700.0
9,node,48522023,48522023,50.1075,19.9955,public_transport,station,transport,5.0,300.0


In [17]:
#krakow_map = get_map_data("Kraków.osm")

In [18]:
get_boundries_points("Kraków")
krakow_boundries = extract_points("Kraków")
krakow_boundries_lat = [i[1] for i in krakow_boundries]
krakow_boundries_lon = [i[2] for i in krakow_boundries]

5771-element Vector{Float64}:
 20.0833859
 20.0833942
 20.0834515
 20.0835144
 20.0837206
 20.0839266
 20.0840815
 20.0841777
 20.0842139
 20.0842551
  ⋮
 20.0955308
 20.095715
 20.0958385
 20.0960187
 20.0960652
 20.0960857
 20.0961556
 20.0961506
 20.0961297

In [19]:
#y = [i[1] for i in krakow_boundries]
#x = [i[2] for i in krakow_boundries]
#
#PlotlyJS.plot(
#    PlotlyJS.scatter(
#        x = x,
#        y = y
#    ),
#
#    config=PlotConfig(scrollZoom=true)
#
#)

In [20]:
ix_krakow = AttractivenessSpatIndex(krakow_df)
center_cracow = LLA(50.061692315544654, 19.939496620660737)
points_cracow = generate_sectors(20,1000,center_cracow,20,
                        krakow_boundries_lat,krakow_boundries_lon)

20×20 Matrix{LLA}:
 LLA(50.0645, 19.9528, 0.0782591)  …  LLA(50.0617, 19.9535, 0.0782384)
 LLA(50.0672, 19.9661, 0.313036)      LLA(50.0617, 19.9674, 0.312954)
 LLA(50.07, 19.9794, 0.704332)        LLA(50.0617, 19.9814, 0.704145)
 LLA(50.0728, 19.9926, 1.25215)       LLA(50.0617, 19.9954, 1.25181)
 LLA(50.0756, 20.0059, 1.95648)       LLA(50.0617, 20.0093, 1.95596)
 LLA(50.0783, 20.0192, 2.81733)    …  LLA(50.0617, 20.0233, 2.81658)
 LLA(50.0811, 20.0325, 3.83469)       LLA(50.0617, 20.0373, 3.83368)
 LLA(50.0839, 20.0458, 5.00858)       LLA(50.0616, 20.0512, 5.00725)
 LLA(50.0866, 20.0591, 6.33898)       LLA(50.0616, 20.0652, 6.33731)
 LLA(50.0894, 20.0724, 7.8259)        LLA(50.0616, 20.0792, 7.82383)
 LLA(50.0922, 20.0857, 9.46933)    …  LLA(50.0616, 20.0931, 9.46684)
 LLA(50.0949, 20.099, 11.2693)        LLA(50.0616, 20.1071, 11.2663)
 LLA(50.0977, 20.1123, 13.2258)       LLA(50.0616, 20.1211, 13.2223)
 LLA(50.1004, 20.1256, 15.3387)       LLA(50.0615, 20.135, 15.3347)
 LLA(50.1032

In [21]:
map_plot = flm.Map(location = [50.061692315544654, 19.939496620660737],
                    zoom_start = 14, control_scale=true)

for sector in 1:size(points_cracow)[1]
    for point in 1:size(points_cracow)[2]
        lt = points_cracow[sector,point].lat
        ln = points_cracow[sector,point].lon
        flm.Marker([lt,ln],popup=1).add_to(map_plot)
    end
end
#for point in krakow_boundries[1:500]
#    lt = point[1]
#    ln = point[2]
#    flm.Marker([lt,ln],popup=1).add_to(map_plot)
#end
map_plot

PyObject <folium.folium.Map object at 0x7fd284fbdb50>